In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 912 ms (started: 2021-08-26 17:14:54 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #7  Aug 26, 2021'

#git push origin master
git push

In [ ]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

# 介绍

这里通常有两种方法可以跨多个设备分配计算：

**数据并行性**，即在多个设备或多台机器上复制单个模型。 他们每个人都处理不同批次的数据，然后合并他们的结果。 这种设置存在许多变体，它们的不同在于不同模型副本如何合并结果，它们是否在每个批次中保持同步，或者它们是否更松散耦合，等等。

**模型并行性**，即单个模型的不同部分在不同设备上运行，一起处理一批数据。 这最适用于具有自然并行架构的模型，例如具有多个分支的模型。

本指南重点介绍数据并行性，尤其是同步数据并行性，其中模型的不同副本在处理每个批次后保持同步。 同步性使模型收敛行为与您在单设备训练中看到的相同。

具体来说，本指南将教您如何使用 `tf.distribute` API 在多个 GPU 上训练 Keras 模型，对您的代码进行最少的更改，在以下两个设置中：
* 在安装在单台机器上的多个 GPU（通常为 2 到 8 个）上（单主机，多设备训练）。 这是研究人员和小规模行业工作流程最常见的设置。
* 在多台机器的集群上，每台机器托管一个或多个 GPU（多工人分布式训练）。 这是大型行业工作流程的良好设置，例如 使用 20-100 个 GPU 在数千万张图像上训练高分辨率图像分类模型。

# 设置

In [3]:
import tensorflow as tf
from tensorflow import keras

time: 6.52 s (started: 2021-08-26 17:17:52 +08:00)


# 单主机多设备同步训练
在此设置中，您有一台带有多个 GPU（通常为 2 到 8 个）的机器。 每个设备都将运行您模型的副本（称为副本）。 为简单起见，在不失一般性的情况下，我们假设我们正在处理 8 个 GPU。

## 这个怎么运作

在训练的每一步：
* 当前批次的数据（称为全局批次）被拆分为 8 个不同的子批次（称为本地批次）。 例如，如果全局批次有 512 个样本，那么 8 个本地批次中的每一个将有 64 个样本。
* 8 个副本中的每一个都独立处理一个本地批次：它们运行前向传递，然后是反向传递，输出与本地批次上模型损失相关的权重梯度。
* 源自局部梯度的权重更新被有效地合并到 8 个副本中。 因为这是在每一步结束时完成的，所以副本始终保持同步。

在实践中，同步更新模型副本权重的过程是在每个单独的权重变量级别处理的。 这是通过镜像变量对象完成的。

## 如何使用它

要使用 Keras 模型进行单主机、多设备同步训练，您可以使用 `tf.distribute.MirroredStrategy` API。 这是它的工作原理：
* 实例化 `MirroredStrategy`，可选择配置您要使用的特定设备（默认情况下，该策略将使用所有可用的 GPU）。
* 使用策略对象打开一个范围，并在此范围内创建您需要的所有包含变量的 Keras 对象。 通常，这意味着在分发范围内创建和编译模型。
* 像往常一样通过 fit() 训练模型。

重要的是，我们建议您使用 `tf.data.Dataset` 对象在多设备或分布式工作流中加载数据。

示意性地，它看起来像这样：

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # 创建变量的一切都应该在策略范围内。
    # 通常这只是模型构建和 `compile()`。
    model = Model(...)
    model.compile(...)

# 在所有可用设备上训练模型。
model.fit(train_dataset, validation_data=val_dataset, ...)

# 在所有可用设备上测试模型。
model.evaluate(test_dataset)

这是一个简单的端到端可运行示例：

In [4]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

time: 947 µs (started: 2021-08-26 17:22:32 +08:00)


In [10]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # 创建变量的一切都应该在策略范围内。
    # 通常这只是模型构建和 `compile()`。
    model = get_compiled_model()

# 在所有可用设备上训练模型。
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# 在所有可用设备上测试模型。
model.evaluate(test_dataset)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Number of devices: 8
Epoch 1/2
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
1563/1563 [==============================] - 68s 38ms/step - loss: 0.2245 - sparse_categorical_accuracy: 0.9324 - val_loss: 0.1195 - val_sparse_categorical_accuracy: 0.9643
Epoch 2/2
313/313 [==============================] - 8s 21ms/step - loss: 0.0872 - sparse_categorical_accuracy: 0.9737


[0.08722639083862305, 0.9736999869346619]

time: 2min 16s (started: 2021-08-26 17:37:47 +08:00)


In [8]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1"])
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # 创建变量的一切都应该在策略范围内。
    # 通常这只是模型构建和 `compile()`。
    model = get_compiled_model()

# 在所有可用设备上训练模型。
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# 在所有可用设备上测试模型。
model.evaluate(test_dataset)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
Epoch 1/2
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
1563/1563 [==============================] - 42s 25ms/step - loss: 0.2256 - sparse_categorical_accuracy: 0.9323 - val_loss: 0.1243 - val_sparse_categorical_accuracy: 0.9622
Epoch 2/2
313/313 [==============================] - 6s 17ms/step - loss: 0.0886 - sparse_categorical_accuracy: 0.9712


[0.0885513424873352, 0.9711999893188477]

time: 1min 26s (started: 2021-08-26 17:30:43 +08:00)


In [9]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy(["GPU:0"])
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # 创建变量的一切都应该在策略范围内。
    # 通常这只是模型构建和 `compile()`。
    model = get_compiled_model()

# 在所有可用设备上训练模型。
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# 在所有可用设备上测试模型。
model.evaluate(test_dataset)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Epoch 1/2
1563/1563 [==============================] - 33s 20ms/step - loss: 0.2239 - sparse_categorical_accuracy: 0.9332 - val_loss: 0.1229 - val_sparse_categorical_accuracy: 0.9631
Epoch 2/2
313/313 [==============================] - 5s 16ms/step - loss: 0.0905 - sparse_categorical_accuracy: 0.9724


[0.09049823880195618, 0.9724000096321106]

time: 1min 9s (started: 2021-08-26 17:32:29 +08:00)


# 使用回调确保容错
使用分布式训练时，您应该始终确保有从故障中恢复的策略（容错）。 处理此问题的最简单方法是将 ModelCheckpoint 回调传递给 fit()，以定期保存您的模型（例如每 100 个批次或每个 epoch）。 然后，您可以从保存的模型重新开始训练。

这是一个简单的例子：

In [12]:
import os
from tensorflow import keras

# 准备一个目录来存储所有检查点。
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


def make_or_restore_model():
    # 要么恢复最新的模型，要么创建一个新的模型
    # 如果没有可用的检查点。
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()

def run_training(epochs=1):
    # Create a MirroredStrategy.
    strategy = tf.distribute.MirroredStrategy(['GPU:0'])

    # 打开策略范围并创建/恢复模型
    with strategy.scope():
        model = make_or_restore_model()

    callbacks = [
        # 这个回调在每个 epoch 保存一个 SavedModel
        # 我们在文件夹名称中包含当前纪元。
        keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir + "/ckpt-{epoch}", save_freq="epoch"
        )
    ]
    model.fit(
        train_dataset,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=val_dataset,
        verbose=2,
    )


# 第一次运行创建模型
run_training(epochs=1)

# 再次调用相同的函数将从我们停止的地方继续
run_training(epochs=1)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Restoring from ./ckpt/ckpt-loss=0.30
1563/1563 - 29s - loss: 0.1419 - sparse_categorical_accuracy: 0.9573 - val_loss: 0.1244 - val_sparse_categorical_accuracy: 0.9662
INFO:tensorflow:Assets written to: ./ckpt/ckpt-1/assets


INFO:tensorflow:Assets written to: ./ckpt/ckpt-1/assets


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Restoring from ./ckpt/ckpt-1
1563/1563 - 28s - loss: 0.1067 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.1169 - val_sparse_categorical_accuracy: 0.9674
INFO:tensorflow:Assets written to: ./ckpt/ckpt-1/assets


INFO:tensorflow:Assets written to: ./ckpt/ckpt-1/assets


time: 1min (started: 2021-08-26 17:42:37 +08:00)


# tf.data 性能提示
在进行分布式训练时，加载数据的效率通常变得至关重要。 这里有一些技巧可以确保您的 `tf.data` 管道尽可能快地运行。

## 关于数据集批处理的注意事项

创建数据集时，请确保使用全局批量大小对其进行批处理。 例如，如果您的 8 个 GPU 中的每一个都能够运行 64 个样本的批次，则您将使用全局批次大小为 512。

## 调用 `dataset.cache()`

如果您对数据集调用 `.cache()` ，则在对数据进行第一次迭代后，其数据将被缓存。 每次后续迭代都将使用缓存数据。 缓存可以在内存中（默认）或您指定的本地文件。

在以下情况下，这可以提高性能：
* 您的数据预计不会随着迭代而变化
* 您正在从远程分布式文件系统读取数据
* 您正在从本地磁盘读取数据，但您的数据适合内存，并且您的工作流程受 IO 限制（例如读取和解码图像文件）。

## 调用 `dataset.prefetch(buffer_size)`

创建数据集后，您几乎应该总是调用` .prefetch(buffer_size)` 。 这意味着您的数据管道将与您的模型异步运行，新样本将被预处理并存储在缓冲区中，而当前批次样本将用于训练模型。 当前批次结束时，下一批将在 GPU 内存中预取。

# 多worker分布式同步训练
## 这个怎么运作

在此设置中，您有多台机器（称为工作器），每台机器上都有一个或多个 GPU。 与单主机训练发生的情况非常相似，每个可用的 GPU 将运行一个模型副本，并且每个副本的变量值在每批之后保持同步。

重要的是，当前的实现假设所有工作人员都拥有相同数量的 GPU（同构集群）。

## 如何使用它
* 建立一个集群（我们在下面提供了一些提示）。
* 在每个 worker 上设置适当的 TF_CONFIG 环境变量。 这告诉工作人员它的角色是什么以及如何与其对等通信。
* 在每个 worker 上，在 MultiWorkerMirroredStrategy 对象的范围内运行您的模型构建和编译代码，类似于我们在单主机训练中所做的。
* 在指定的评估器机器上运行评估代码。

## 设置集群

首先，建立一个集群（机器的集合）。 每台机器都应该单独设置，以便能够运行您的模型（通常，每台机器将运行相同的 Docker 映像）并能够访问您的数据源（例如 GCS）。

集群管理超出了本指南的范围。 这是帮助您入门的文档。 你也可以看看 Kubeflow。

## 设置 `TF_CONFIG` 环境变量

虽然在每个 worker 上运行的代码几乎与单主机工作流中使用的代码相同（除了不同的 `tf.distribute` 策略对象），但单主机工作流和多工作流工作流之间的一个显着区别是 您需要在集群中运行的每台机器上设置一个 `TF_CONFIG` 环境变量。

`TF_CONFIG` 环境变量是一个 `JSON` 字符串，它指定：
* 集群配置，而构成集群的机器的地址和端口列表
* 工人的“任务”，这是这台特定机器在集群中必须扮演的角色。

`TF_CONFIG` 的一个例子是：

In [ ]:
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})

在多工人同步训练设置中，机器的有效角色（任务类型）是“工人”和“评估者”。

例如，如果您有 8 台机器，每台机器有 4 个 GPU，那么您可以有 7 个工作人员和一名评估员。
* 工人训练模型，每个人处理全局批次的子批次。
* 其中一个工作人员（工作人员 0）将担任“首席”，一种特殊类型的工作人员，负责保存日志和检查点以供以后重用（通常到云存储位置）。
* 评估器运行一个连续循环，加载首席工人保存的最新检查点，对其运行评估（与其他工人异步）并写入评估日志（例如 TensorBoard 日志）。

## 在每个工人上运行代码

您将对每个工人（包括负责人）运行培训代码，并在评估者身上运行评估代码。

除了使用 `MultiWorkerMirroredStrategy` 而不是 `MirroredStrategy` 之外，训练代码与您在单主机设置中使用的代码基本相同。

每个工作人员将运行相同的代码（减去下面注释中解释的差异），包括相同的回调。

> 注意：保存模型检查点或日志的回调应该为每个工作人员保存到不同的目录。标准做法是所有工作人员都应保存到本地磁盘（通常是临时的），但工作人员 0 除外，这会将 TensorBoard 日志检查点保存到云存储位置以供以后访问和重用。

评估器将简单地使用 MirroredStrategy（因为它在单台机器上运行并且不需要与其他机器通信）并调用 `model.evaluate()`。它将首席工人保存的最新检查点加载到云存储位置，并将评估日志保存到与首席日志相同的位置。

## 示例：在多线程设置中运行的代码
在首席（工人 0）上：

In [ ]:
# Set TF_CONFIG
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})


# Open a strategy scope and create/restore the model.
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
with strategy.scope():
    model = make_or_restore_model()

callbacks = [
    # This callback saves a SavedModel every 100 batches
    keras.callbacks.ModelCheckpoint(filepath='path/to/cloud/location/ckpt',
                                    save_freq=100),
    keras.callbacks.TensorBoard('path/to/cloud/location/tb/')
]
model.fit(train_dataset,
          callbacks=callbacks,
          ...)

关于其他工人：

In [ ]:
# Set TF_CONFIG
worker_index = 1  # For instance
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': worker_index}
})


# 打开策略范围并创建/恢复模型。
# 你可以从酋长保存的检查点恢复。
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
with strategy.scope():
    model = make_or_restore_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath='local/path/ckpt', save_freq=100),
    keras.callbacks.TensorBoard('local/path/tb/')
]
model.fit(train_dataset,
          callbacks=callbacks,
          ...)

在评估者上：

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = make_or_restore_model()  # Restore from the checkpoint saved by the chief.

results = model.evaluate(val_dataset)
# 然后，在共享位置记录结果，写入 TensorBoard 日志等